1/ Importe ce dataset de tweets dans un DataFrame.

2/ Conserve uniquement les tweets positifs et négatifs (donc tu exclus les neutrals). Quel est le pourcentage de tweets positifs/négatifs ?

3/ Crée une fonction que tu appelleras clean qui prend pour paramètre une phrase (donc un texte str) et qui retourne un texte (str) de tokens après application d'un stemmer ou d'un lemmatizer, séparés par des espaces.
Tu peux tester ta fonction avec cette phrase, elle doit retourner quelque chose ressemblant à ce résultat :

--> clean("You are better when I am well.")

-->"you be well when I be well ."

4/ Récupère la liste des stopwords anglais depuis NLTK, et copie-la dans une liste "stopwordsenglish". Complète ta fonction clean pour qu'elle supprime la ponctuation et les stopwords.

5/ Applique cette fonction clean à la colonne text de ton DataFrame. Stocke le résultat dans une nouvelle colonne clean du DataFrame. (Le traitement peut durer 2 ou 3 minutes)

Tu dois maintenant avoir un DataFrame qui ressemble à celui-ci (moins les pronoms, la ponctuation et peut-être d'autres mots en fonction des stopwords que tu as nettoyé) :

6/ Copie la colonne clean dans une Serie X, et la colonne sentiment dans une Serie y. Applique un train-test split avec la taille du jeu d'entrainement à 0.75 avec le random_state = 32.

7/ Applique un CountVectorizer et entraine des modèles de classification.

8/ Applique un TfidfVectorizer et entraine des modèles de classification.

9/ Compare les scores, quels paramètres permettent d'avoir les meilleurs scores ?

Bonus : maintenant c'est à toi d'améliorer ton modèle :

- En cherchant des paramètres de modèles : par gridsearch et crossvalidation par exemple ;

- En changeant la préparation du texte : par exemple certaines ponctuations peuvent aider le modèle, le point d'exclamation notamment.


In [3]:

import pandas as pd
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\aurel\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\aurel\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\aurel\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\aurel\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\aurel\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

True

In [4]:
df_teewts = pd.read_csv("https://github.com/murpi/wilddata/raw/master/quests/tweets.zip")

In [5]:
df_teewts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27480 entries, 0 to 27479
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27480 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27480 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


In [6]:
df_teewts.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [7]:
df_0 = df_teewts[~df_teewts['sentiment'].str.contains("neutral")]
df_0['sentiment'].value_counts(normalize=True).mul(100).round(2)

sentiment
positive    52.45
negative    47.55
Name: proportion, dtype: float64

In [8]:
import spacy

def clean(text):
    nlp = spacy.load('en_core_web_sm')
    list_mots = []
    for mot in nlp(text):
        list_mots.append(mot.lemma_)
    text_clean = " ".join(list_mots) 
    return text_clean

clean("You are better when I am well.")


'you be well when I be well .'

In [11]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
nlp = spacy.load('en_core_web_sm')

def clean(text):
    article_1 = tokenizer.tokenize(text.lower())
    tokens_clean = []
    for words in article_1:
        if words not in nltk.corpus.stopwords.words("english"):
            tokens_clean.append(words)
    new_text = " ".join(tokens_clean) 
    list_mots = []
    for mot in nlp(new_text):
        list_mots.append(mot.lemma_)
    text_clean = " ".join(list_mots) 
    return text_clean

clean("You are better when I am well.")

'well well'

In [12]:
df_teewts['clean_text'] = df_teewts['text'].apply(clean)

In [13]:
X = df_teewts['clean_text']
y = df_teewts['sentiment']

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

vectorizer = CountVectorizer()

X_train,X_test, y_train, y_test = train_test_split(X, y, random_state=32, train_size=0.75)

vectorizer.fit(X_train, y_train)

X_train_CV = vectorizer.transform(X_train)
X_test_CV = vectorizer.transform(X_test)


In [15]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression().fit(X_train_CV,y_train)

model.score(X_train_CV, y_train), model.score(X_test_CV, y_test)

C:\Users\aurel\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.8962639495390587, 0.6940320232896652)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

X_train,X_test, y_train, y_test = train_test_split(X, y, random_state=32, train_size=0.75)

tfidf.fit(X_train, y_train)

X_train_TF = tfidf.transform(X_train)
X_test_TF = tfidf.transform(X_test)

In [17]:

model = LogisticRegression().fit(X_train_TF,y_train)

model.score(X_train_TF, y_train), model.score(X_test_TF, y_test)

(0.8133915574963609, 0.6905385735080058)